# AlexNet

In [ ]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

# AlexNet Data augmentation

Same data augmentation as described in the original AlexNet paper.

### Image translations and horizontal reflections

Input image sizes before data augmentation are (256, 256).

Random (224, 224) patches are extracted along with their horizontal reflections.

**How does that transform work on multiple items?** They work on multiple items through use of the data loader. By using transforms, you are specifying what should happen to a single emission of data (e.g., batch_size=1). The data loader takes your specified batch_size and makes n calls to the ```__getitem__``` method in the torch data set, applying the transform to each sample sent into training/validation. It then collates n samples into your batch size emitted from the data loader. (Source: https://stackoverflow.com/questions/66370250/how-does-pytorch-dataloader-interact-with-a-pytorch-dataset-to-transform-batches) 


In [ ]:
# the transformer for AlexNet
transformer=transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],
                        [0.5,0.5,0.5])
])

### Altering intensities of RGB channels

AlexNet also alters the RGB intensities in training images. After finding the principle components (after PCA) in the whole image dataset, they perform eigenvector x eigenvalue x random variable and add this to each input image.

However, this only reduces error by 1%, it isn't common practice and hasn't been seen in other implementations of AlexNet - is it necessary?


In [ ]:
# check these pca values from imagenet? Or find for my own dataset?
__imagenet_pca = {
    'eigval': torch.Tensor([0.2175, 0.0188, 0.0045]),
    'eigvec': torch.Tensor([
        [-0.5675,  0.7192,  0.4009],
        [-0.5808, -0.0045, -0.8140],
        [-0.5836, -0.6948,  0.4203],
    ])
}

# Lighting data augmentation take from here - https://github.com/eladhoffer/convNet.pytorch/blob/master/preprocess.py
class Lighting(object):
    """Lighting noise(AlexNet - style PCA - based noise)"""

    def __init__(self, alphastd, eigval, eigvec):
        self.alphastd = alphastd
        self.eigval = eigval
        self.eigvec = eigvec

    def __call__(self, img):
        if self.alphastd == 0:
            return img

        alpha = img.new().resize_(3).normal_(0, self.alphastd)
        rgb = self.eigvec.type_as(img).clone()\
            .mul(alpha.view(1, 3).expand(3, 3))\
            .mul(self.eigval.view(1, 3).expand(3, 3))\
            .sum(1).squeeze()
        return img.add(rgb.view(3, 1, 1).expand_as(img))

transformer = transforms.Compose([
            transforms.RandomResizedCrop(256),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            Lighting(0.1, __imagenet_pca['eigval'], __imagenet_pca['eigvec']),
            transforms.Normalize([0.5,0.5,0.5],
                        [0.5,0.5,0.5])
        ])

### Using an augmentation policy

https://pytorch.org/vision/stable/generated/torchvision.transforms.AutoAugment.html#torchvision.transforms.AutoAugment

# The model

The original AlexNet as written in the paper can be found here: https://github.com/dansuh17/alexnet-pytorch/blob/master/model.py 

The PyTorch implementation (found here: https://github.com/pytorch/vision/blob/main/torchvision/models/alexnet.py) uses a single GPU version of the original AlexNet (https://github.com/pytorch/vision/pull/463)